# 本文将用于从Tensorflow/TFLite中提取Weights

## For mobilenetv1_128_0.25模型

## 1.从Tensorflow(.h5 File)中提取Conv2D的Weights

In [1]:
import numpy as np

#以下为一个提取Tensorflow Conv2d卷积weights+bias的函数，会以txt保存在程序目录下
def OpenNNA_Conv2d_GetWeights_From_TF(Layer_Number,Para,Weights_Index,Bias_Index,precision):
    data_t = precision#"%8.6f"
    weights_file_path = "./"+str(Layer_Number)+"_conv2d_"+str(Layer_Number)+"_weights.txt"
    Weights   = weights[Weights_Index]
    bias_file_path = "./"+str(Layer_Number)+"_conv2d_"+str(Layer_Number)+"_bias.txt"
    Bias = weights[Bias_Index]

    Conv2d_Kernel_size_row = Para[0] #卷积核大小
    Conv2d_Kernel_size_col = Para[1] #卷积核大小
    Conv2d_Kernel_channel = Para[2] #卷积核通道
    Conv2d_Kernel_number = Para[3] #卷积核数量
    Conv2d_bias_number = Para[4]  #偏置数量

    #tensorflow hwc -> chw
    loadData0 = np.swapaxes(Weights, 0, 2)
    loadData1 = np.swapaxes(loadData0, 1, 3)
    loadData2 = np.swapaxes(loadData1, 0, 1)
    txtfile0 = open(weights_file_path, 'w',encoding='UTF-8')
    for i in range(0,Conv2d_Kernel_number):
        txtfile0.write("\t{\n")
        for j in range(0,Conv2d_Kernel_channel):
            txtfile0.write("\t\t{\n")
            for k in range(0,Conv2d_Kernel_size_row):
                txtfile0.write("\t\t\t{")
                for w in range(0,Conv2d_Kernel_size_col):
                   txtfile0.write(data_t%(loadData2[i][j][k][w])+", ") 
                txtfile0.write("},\n")  
            txtfile0.write("\t\t},\n")
        txtfile0.write("\t},\n\n")
    txtfile0.close()
    #Bias
    txtfile1 = open(bias_file_path, 'w',encoding='UTF-8')
    txtfile1.write("\t{")
    for i in range(0,Conv2d_bias_number):
        txtfile1.write(data_t%(Bias[i])+", ")
    txtfile1.write("}")
    txtfile1.close()
    print("OpenNNA: Conv2d Layer:%d Weights[%d]+Bias[%d] Get Success!"%(Layer_Number,Weights_Index,Bias_Index))

## 2.从Tensorflow(.h5 File)中提取Dense的Weights

In [2]:
import numpy as np

#以下为一个提取Tensorflow Dense卷积weights+bias的函数，会以txt保存在程序目录下
def OpenNNA_Dense_GetWeights_From_TF(Layer_Number,Para,Weights_Index,Bias_Index,precision):
    data_t = precision#"%8.6f"
    weights_file_path = "./"+str(Layer_Number)+"_dense_"+str(Layer_Number)+"_weights.txt"
    Weights   = weights[Weights_Index]
    bias_file_path = "./"+str(Layer_Number)+"_dense_"+str(Layer_Number)+"_bias.txt"
    Bias = weights[Bias_Index]
    #tensorflow默认的weights存储格式为HWC，在这里需要转为OpenNNA支持的CHW
    Dense_Input_Channel = Para[0] #神经元输入channel c
    Dense_Input_Row = Para[1] #神经元输入row h
    Dense_Input_Col = Para[2] #神经元输入col w
    Dense_Units = Para[3] #神经元数量
    Dense_bias_number = Para[4]  #偏置数量

    dense1_weights = np.swapaxes(Weights,0,1)
    txtfile_1 = open(weights_file_path, 'w', encoding='UTF-8')
    txtfile_1.write("\n{")
    for i in range(0,Dense_Units):
        txtfile_1.write("\n\t{")
        for j in range(0,Dense_Input_Channel):#c
            for k in range(0,Dense_Input_Row):#h
                for l in range(0,Dense_Input_Col):#w
                    txtfile_1.write(data_t%dense1_weights[i][j+l*Dense_Input_Channel+k*Dense_Input_Channel*Dense_Input_Col]+",")
        txtfile_1.write("},\n")
    txtfile_1.write("\n}")
    txtfile_1.close()

    #提取第一层Dense的bias
    txtfile_1 = open(bias_file_path, 'w', encoding='UTF-8')
    txtfile_1.write("\n{")
    for i in range(0,Dense_bias_number):
        txtfile_1.write(data_t%Bias[i]+",")
    txtfile_1.write("}")
    txtfile_1.close()
    print("OpenNNA: Dense Layer:%d Weights[%d]+Bias[%d] Get Success!"%(Layer_Number,Weights_Index,Bias_Index))

## 3.加载训练好的模型文件(.h5 File)

In [3]:
from keras.models import load_model

model = load_model('Conv(OpenNNA-Paper)-Mnist-Int8.h5')

## 4.加载TFLite

In [1]:
import tensorflow as tf
tflite_model_name = 'mobilenet_v1_0.25_128_quant.tflite'
'''
Create interpreter, allocate tensors
'''
tflite_interpreter = tf.lite.Interpreter(model_path=tflite_model_name)
tflite_interpreter.allocate_tensors()

'''
Check input/output details
'''
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])
print("\n\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])
print()
'''
Run prediction (optional), input_array has input's shape and dtype
'''
'''
tflite_interpreter.set_tensor(input_details[0]['index'], input_array)
tflite_interpreter.invoke()
output_array = tflite_interpreter.get_tensor(output_details[0]['index'])
'''
'''
This gives a list of dictionaries. 
'''
tensor_details = tflite_interpreter.get_tensor_details()

for dict in tensor_details:
    i = dict['index']
    tensor_name = dict['name']
    scales = dict['quantization_parameters']['scales']
    zero_points = dict['quantization_parameters']['zero_points']
    tensor = tflite_interpreter.tensor(i)()

    print(i, "|",type, "|",tensor_name, "|","Scales:",scales, "|","zero_points",zero_points, "|",tensor.shape)
    #print(tensor)
    '''
    See note below
    '''
#print("\n\n\n")
#print(tflite_interpreter.tensor(2)().shape)#tensor_details排布:List套字典

== Input details ==
name: input
shape: [  1 128 128   3]
type: <class 'numpy.uint8'>


== Output details ==
name: MobilenetV1/Predictions/Reshape_1
shape: [   1 1001]
type: <class 'numpy.uint8'>

0 | <class 'type'> | MobilenetV1/Logits/AvgPool_1a/AvgPool | Scales: [0.02352848] | zero_points [0] | (1, 1, 1, 256)
1 | <class 'type'> | MobilenetV1/Logits/Conv2d_1c_1x1/BiasAdd | Scales: [0.13083284] | zero_points [96] | (1, 1, 1, 1001)
2 | <class 'type'> | MobilenetV1/Logits/Conv2d_1c_1x1/Conv2D_bias | Scales: [0.00013035] | zero_points [0] | (1001,)
3 | <class 'type'> | MobilenetV1/Logits/Conv2d_1c_1x1/weights_quant/FakeQuantWithMinMaxVars | Scales: [0.00554029] | zero_points [94] | (1001, 1, 1, 256)
4 | <class 'type'> | MobilenetV1/Logits/SpatialSqueeze | Scales: [0.13083284] | zero_points [96] | (1, 1001)
5 | <class 'type'> | MobilenetV1/Logits/SpatialSqueeze_shape | Scales: [] | zero_points [] | (2,)
6 | <class 'type'> | MobilenetV1/MobilenetV1/Conv2d_0/Conv2D_Fold_bias | Scales: [6.939

## 5.从Tensorflow(.tflite File)中提取Dense的Weights

In [6]:
import numpy as np

#以下为一个提取Tensorflow Dense卷积weights+bias的函数，会以txt保存在程序目录下
def OpenNNA_Dense_GetWeights_From_TFLite(Layer_Number,Para,Weights_Index,Bias_Index,precision):
    tflite_interpreter = tf.lite.Interpreter(model_path=tflite_model_name)
    tflite_interpreter.allocate_tensors()
    data_t = precision#"%8.6f"
    weights_file_path = "./"+str(Layer_Number)+"_dense_"+str(Layer_Number)+"_weights.txt"
    Weights   = tflite_interpreter.tensor(Weights_Index)()
    bias_file_path = "./"+str(Layer_Number)+"_dense_"+str(Layer_Number)+"_bias.txt"
    Bias = tflite_interpreter.tensor(Bias_Index)()
    #tflite默认的weights存储格式为HWC，在这里需要转为OpenNNA支持的CHW
    Dense_Input_Channel = Para[0] #神经元输入channel c
    Dense_Input_Row = Para[1] #神经元输入row h
    Dense_Input_Col = Para[2] #神经元输入col w
    Dense_Units = Para[3] #神经元数量
    Dense_bias_number = Para[4]  #偏置数量
    #print(Weights.shape)
    dense1_weights = Weights#np.swapaxes(Weights,0,1)
    #dense1_weights = np.swapaxes(Weights,0,1)
    print(dense1_weights.shape)
    txtfile_1 = open(weights_file_path, 'w', encoding='UTF-8')
    txtfile_1.write("\n{")
    for i in range(0,Dense_Units):
        txtfile_1.write("\n\t{")
        for j in range(0,Dense_Input_Channel):#c
            for k in range(0,Dense_Input_Row):#h
                for l in range(0,Dense_Input_Col):#w
                    # HWC -> CHW
                    txtfile_1.write(data_t%dense1_weights[i][j+l*Dense_Input_Channel+k*Dense_Input_Channel*Dense_Input_Col]+",")
                    #CHW -> CHW
                    #txtfile_1.write(data_t%dense1_weights[i][l+k*Dense_Input_Col+j*Dense_Input_Col*Dense_Input_Row]+",")
        txtfile_1.write("},\n")
    txtfile_1.write("\n}")
    txtfile_1.close()

    #提取第一层Dense的bias
    txtfile_1 = open(bias_file_path, 'w', encoding='UTF-8')
    txtfile_1.write("\n{")
    for i in range(0,Dense_bias_number):
        txtfile_1.write(data_t%Bias[i]+",")
    txtfile_1.write("}")
    txtfile_1.close()
    print("OpenNNA: Dense Layer:%d Weights[%d]+Bias[%d] Get Success!"%(Layer_Number,Weights_Index,Bias_Index))

## 6.从Tensorflow(.tflite File)中提取Conv2D的Weights

In [22]:
import numpy as np

#以下为一个提取Tensorflow Conv2d卷积weights+bias的函数，会以txt保存在程序目录下
def OpenNNA_Conv2d_GetWeights_From_TFLite(Layer_Number,Para,Weights_Index,Bias_Index,precision):
    tflite_interpreter = tf.lite.Interpreter(model_path=tflite_model_name)
    tflite_interpreter.allocate_tensors()
    data_t = precision#"%8.6f"
    weights_file_path = "./"+str(Layer_Number)+"_conv2d_"+str(Layer_Number)+"_weights.txt"
    Weights   = tflite_interpreter.tensor(Weights_Index)()
    bias_file_path = "./"+str(Layer_Number)+"_conv2d_"+str(Layer_Number)+"_bias.txt"
    Bias = tflite_interpreter.tensor(Bias_Index)()

    Conv2d_Kernel_size_row = Para[0] #卷积核大小
    Conv2d_Kernel_size_col = Para[1] #卷积核大小
    Conv2d_Kernel_channel = Para[2] #卷积核通道
    Conv2d_Kernel_number = Para[3] #卷积核数量
    Conv2d_bias_number = Para[4]  #偏置数量

    #tensorflow hwc -> chw
    loadData0 = np.swapaxes(Weights, 2, 3)
    loadData2 = np.swapaxes(loadData0, 1, 2)
    #loadData2 = np.swapaxes(loadData1, 0, 1)
    txtfile0 = open(weights_file_path, 'w',encoding='UTF-8')
    for i in range(0,Conv2d_Kernel_number):
        txtfile0.write("\t{\n")
        for j in range(0,Conv2d_Kernel_channel):
            txtfile0.write("\t\t{\n")
            for k in range(0,Conv2d_Kernel_size_row):
                txtfile0.write("\t\t\t{")
                for w in range(0,Conv2d_Kernel_size_col):
                   txtfile0.write(data_t%(loadData2[i][j][k][w])+", ") 
                txtfile0.write("},\n")  
            txtfile0.write("\t\t},\n")
        txtfile0.write("\t},\n\n")
    txtfile0.close()
    #Bias
    txtfile1 = open(bias_file_path, 'w',encoding='UTF-8')
    txtfile1.write("\t{")
    for i in range(0,Conv2d_bias_number):
        txtfile1.write(data_t%(Bias[i])+", ")
    txtfile1.write("}")
    txtfile1.close()
    print("OpenNNA: Conv2d Layer:%d Weights[%d]+Bias[%d] Get Success!"%(Layer_Number,Weights_Index,Bias_Index))

## 7.从Tensorflow(.tflite File)中提取Depthwise Conv2D的Weights

In [23]:
import numpy as np

#以下为一个提取Tensorflow Conv2d卷积weights+bias的函数，会以txt保存在程序目录下
def OpenNNA_DWConv2d_GetWeights_From_TFLite(Layer_Number,Para,Weights_Index,Bias_Index,precision):
    tflite_interpreter = tf.lite.Interpreter(model_path=tflite_model_name)
    tflite_interpreter.allocate_tensors()
    data_t = precision#"%8.6f"
    weights_file_path = "./"+str(Layer_Number)+"_dwconv2d_"+str(Layer_Number)+"_weights.txt"
    Weights   = tflite_interpreter.tensor(Weights_Index)()
    bias_file_path = "./"+str(Layer_Number)+"_dwconv2d_"+str(Layer_Number)+"_bias.txt"
    Bias = tflite_interpreter.tensor(Bias_Index)()

    Conv2d_Kernel_size_row = Para[0] #卷积核大小
    Conv2d_Kernel_size_col = Para[1] #卷积核大小
    Conv2d_Kernel_channel = Para[2] #卷积核通道
    Conv2d_Kernel_number = Para[3] #卷积核数量
    Conv2d_bias_number = Para[4]  #偏置数量

    #tensorflow hwc -> chw
    loadData0 = np.swapaxes(Weights, 2, 3)
    loadData2 = np.swapaxes(loadData0, 1, 2)
    #loadData2 = np.swapaxes(loadData1, 0, 1)
    txtfile0 = open(weights_file_path, 'w',encoding='UTF-8')
    for i in range(0,Conv2d_Kernel_number):
        txtfile0.write("\t{\n")
        for j in range(0,Conv2d_Kernel_channel):
            txtfile0.write("\t\t{\n")
            for k in range(0,Conv2d_Kernel_size_row):
                txtfile0.write("\t\t\t{")
                for w in range(0,Conv2d_Kernel_size_col):
                   txtfile0.write(data_t%(loadData2[i][j][k][w])+", ") 
                txtfile0.write("},\n")  
            txtfile0.write("\t\t},\n")
        txtfile0.write("\t},\n\n")
    txtfile0.close()
    #Bias
    txtfile1 = open(bias_file_path, 'w',encoding='UTF-8')
    txtfile1.write("\t{")
    for i in range(0,Conv2d_bias_number):
        txtfile1.write(data_t%(Bias[i])+", ")
    txtfile1.write("}")
    txtfile1.close()
    print("OpenNNA: Depthwise Conv2d Layer:%d Weights[%d]+Bias[%d] Get Success!"%(Layer_Number,Weights_Index,Bias_Index))

In [24]:
#与Tensorflow H5 weights&bias的index需要手动printf检查不同， tflite每层weights&bias的index就是Netron所显示的location
OpenNNA_Conv2d_GetWeights_From_TFLite(0,[3,3,3,8,8],8,6,"%d")#第0层Conv2d

OpenNNA_DWConv2d_GetWeights_From_TFLite(1,[3,3,8,1,8],35,34,"%d")#第1层Depthwise Conv2d
OpenNNA_Conv2d_GetWeights_From_TFLite(2,[1,1,8,16,16],38,36,"%d")#第2层Conv2d

OpenNNA_DWConv2d_GetWeights_From_TFLite(3,[3,3,16,1,16],41,40,"%d")#第3层Depthwise Conv2d
OpenNNA_Conv2d_GetWeights_From_TFLite(4,[1,1,16,32,32],44,42,"%d")#第4层Conv2d

OpenNNA_DWConv2d_GetWeights_From_TFLite(5,[3,3,32,1,32],47,46,"%d")#第5层Depthwise Conv2d
OpenNNA_Conv2d_GetWeights_From_TFLite(6,[1,1,32,32,32],50,48,"%d")#第6层Conv2d

OpenNNA_DWConv2d_GetWeights_From_TFLite(7,[3,3,32,1,32],53,52,"%d")#第7层Depthwise Conv2d
OpenNNA_Conv2d_GetWeights_From_TFLite(8,[1,1,32,64,64],56,54,"%d")#第8层Conv2d

OpenNNA_DWConv2d_GetWeights_From_TFLite(9,[3,3,64,1,64],59,58,"%d")#第9层Depthwise Conv2d
OpenNNA_Conv2d_GetWeights_From_TFLite(10,[1,1,64,64,64],62,60,"%d")#第10层Conv2d

OpenNNA_DWConv2d_GetWeights_From_TFLite(11,[3,3,64,1,64],65,64,"%d")#第11层Depthwise Conv2d
OpenNNA_Conv2d_GetWeights_From_TFLite(12,[1,1,64,128,128],68,66,"%d")#第12层Conv2d

OpenNNA_DWConv2d_GetWeights_From_TFLite(13,[3,3,128,1,128],71,70,"%d")#第13层Depthwise Conv2d
OpenNNA_Conv2d_GetWeights_From_TFLite(14,[1,1,128,128,128],74,72,"%d")#第14层Conv2d

OpenNNA_DWConv2d_GetWeights_From_TFLite(15,[3,3,128,1,128],77,76,"%d")#第15层Depthwise Conv2d
OpenNNA_Conv2d_GetWeights_From_TFLite(16,[1,1,128,128,128],80,78,"%d")#第16层Conv2d

OpenNNA_DWConv2d_GetWeights_From_TFLite(17,[3,3,128,1,128],83,82,"%d")#第17层Depthwise Conv2d
OpenNNA_Conv2d_GetWeights_From_TFLite(18,[1,1,128,128,128],86,84,"%d")#第18层Conv2d

OpenNNA_DWConv2d_GetWeights_From_TFLite(19,[3,3,128,1,128],11,10,"%d")#第19层Depthwise Conv2d
OpenNNA_Conv2d_GetWeights_From_TFLite(20,[1,1,128,128,128],14,12,"%d")#第20层Conv2d

OpenNNA_DWConv2d_GetWeights_From_TFLite(21,[3,3,128,1,128],17,16,"%d")#第21层Depthwise Conv2d
OpenNNA_Conv2d_GetWeights_From_TFLite(22,[1,1,128,128,128],20,18,"%d")#第22层Conv2d

OpenNNA_DWConv2d_GetWeights_From_TFLite(23,[3,3,128,1,128],23,22,"%d")#第23层Depthwise Conv2d
OpenNNA_Conv2d_GetWeights_From_TFLite(24,[1,1,128,256,256],26,24,"%d")#第24层Conv2d

OpenNNA_DWConv2d_GetWeights_From_TFLite(25,[3,3,256,1,256],29,28,"%d")#第25层Depthwise Conv2d
OpenNNA_Conv2d_GetWeights_From_TFLite(26,[1,1,256,256,256],32,30,"%d")#第26层Conv2d

#第27层AvgPool

OpenNNA_Conv2d_GetWeights_From_TFLite(28,[1,1,256,1001,1001],3,2,"%d")#第28层Conv2d

#Output:1001


#OpenNNA_Dense_GetWeights_From_TFLite(10,[32,6,6,64,64],11,12,"%d")#第10层Dense
#OpenNNA_Dense_GetWeights_From_TFLite(11,[1,1,64,10,10],13,14,"%d")#第11层Dense

OpenNNA: Conv2d Layer:0 Weights[8]+Bias[6] Get Success!
OpenNNA: Depthwise Conv2d Layer:1 Weights[35]+Bias[34] Get Success!
OpenNNA: Conv2d Layer:2 Weights[38]+Bias[36] Get Success!
OpenNNA: Depthwise Conv2d Layer:3 Weights[41]+Bias[40] Get Success!
OpenNNA: Conv2d Layer:4 Weights[44]+Bias[42] Get Success!
OpenNNA: Depthwise Conv2d Layer:5 Weights[47]+Bias[46] Get Success!
OpenNNA: Conv2d Layer:6 Weights[50]+Bias[48] Get Success!
OpenNNA: Depthwise Conv2d Layer:7 Weights[53]+Bias[52] Get Success!
OpenNNA: Conv2d Layer:8 Weights[56]+Bias[54] Get Success!
OpenNNA: Depthwise Conv2d Layer:9 Weights[59]+Bias[58] Get Success!
OpenNNA: Conv2d Layer:10 Weights[62]+Bias[60] Get Success!
OpenNNA: Depthwise Conv2d Layer:11 Weights[65]+Bias[64] Get Success!
OpenNNA: Conv2d Layer:12 Weights[68]+Bias[66] Get Success!
OpenNNA: Depthwise Conv2d Layer:13 Weights[71]+Bias[70] Get Success!
OpenNNA: Conv2d Layer:14 Weights[74]+Bias[72] Get Success!
OpenNNA: Depthwise Conv2d Layer:15 Weights[77]+Bias[76] G

## 8.从Tensorflow(.tflite File)中提取Dense的Weights,并生成.h文件
与保存到txt文件不同，保存到.h文件将会同时保存当前层的scale和zero point信息

In [7]:
import numpy as np

#以下为一个提取Tensorflow Dense卷积weights+bias的函数，会以txt保存在程序目录下
def OpenNNA_Dense_GetWeights_From_TFLite_Save_HFile(Layer_Number,Para,Weights_Index,Bias_Index,precision):
    tflite_interpreter = tf.lite.Interpreter(model_path=tflite_model_name)
    tflite_interpreter.allocate_tensors()
    data_t = precision#"%8.6f"
    weights_file_path = "./L"+str(Layer_Number)+"_Dense_Weights"+".h"
    Weights   = tflite_interpreter.tensor(Weights_Index)()
    bias_file_path = "./L"+str(Layer_Number)+"_Dense_Bias"+".h"
    Bias = tflite_interpreter.tensor(Bias_Index)()
    #tflite默认的weights存储格式为HWC，在这里需要转为OpenNNA支持的CHW
    Dense_Input_Channel = Para[0] #神经元输入channel c
    Dense_Input_Row = Para[1] #神经元输入row h
    Dense_Input_Col = Para[2] #神经元输入col w
    Dense_Units = Para[3] #神经元数量
    Dense_bias_number = Para[4]  #偏置数量
    #print(Weights.shape)
    dense1_weights = Weights#np.swapaxes(Weights,0,1)
    #dense1_weights = np.swapaxes(Weights,0,1)
    print(dense1_weights.shape)
    txtfile_1 = open(weights_file_path, 'w', encoding='UTF-8')
    #.h File Header
    txtfile_1.write("#ifndef __L%d_DENSE_WEIGHTS_H__\n"%(Layer_Number))
    txtfile_1.write("#define __L%d_DENSE_WEIGHTS_H__\n"%(Layer_Number))
    #数组
    txtfile_1.write("const Weights_t L%d_Dense_Weights[1][%d][%d]={\n"%(Layer_Number,Dense_Units,(Dense_Input_Channel*Dense_Input_Row*Dense_Input_Col)))
    #Weights
    txtfile_1.write("\n\t{")
    for i in range(0,Dense_Units):
        txtfile_1.write("\n\t\t{")
        for j in range(0,Dense_Input_Channel):#c
            for k in range(0,Dense_Input_Row):#h
                for l in range(0,Dense_Input_Col):#w
                    # HWC -> CHW
                    txtfile_1.write(data_t%dense1_weights[i][j+l*Dense_Input_Channel+k*Dense_Input_Channel*Dense_Input_Col]+",")
                    #CHW -> CHW
                    #txtfile_1.write(data_t%dense1_weights[i][l+k*Dense_Input_Col+j*Dense_Input_Col*Dense_Input_Row]+",")
        txtfile_1.write("},\n")
    txtfile_1.write("\n\t}")
    #.h File End
    txtfile_1.write("\n};")
    txtfile_1.write("\n#endif")
    txtfile_1.close()

    #提取第一层Dense的bias
    txtfile_1 = open(bias_file_path, 'w', encoding='UTF-8')
    #.h File Header
    txtfile_1.write("#ifndef __L%d_DENSE_BIAS_H__\n"%(Layer_Number))
    txtfile_1.write("#define __L%d_DENSE_BIAS_H__\n"%(Layer_Number))
    #数组
    txtfile_1.write("const Bias_t L%d_Dense_Bias[%d]={\n"%(Layer_Number,Dense_Units))
    #Bias
    txtfile_1.write("\n\t")
    for i in range(0,Dense_bias_number):
        txtfile_1.write(data_t%Bias[i]+",")
    txtfile_1.write("\n};")
    txtfile_1.write("\n#endif")
    txtfile_1.close()
    print("OpenNNA: Dense Layer:%d Weights[%d]+Bias[%d] Get Success!"%(Layer_Number,Weights_Index,Bias_Index))

## 9.从Tensorflow(.tflite File)中提取Depthwise Conv2D的Weights,并生成.h文件
与保存到txt文件不同，保存到.h文件将会同时保存当前层的scale和zero point信息

In [2]:
import numpy as np

#以下为一个提取Tensorflow Conv2d卷积weights+bias的函数，会以txt保存在程序目录下
def OpenNNA_DWConv2d_GetWeights_From_TFLite_Save_HFile(Layer_Number,Para,Weights_Index,Bias_Index,precision):
    tflite_interpreter = tf.lite.Interpreter(model_path=tflite_model_name)
    tflite_interpreter.allocate_tensors()
    data_t = precision#"%8.6f"
    weights_file_path = "./L"+str(Layer_Number)+"_DwConv2d_Weights"+".h"
    Weights   = tflite_interpreter.tensor(Weights_Index)()
    bias_file_path = "./L"+str(Layer_Number)+"_DwConv2d_Bias"+".h"
    Bias = tflite_interpreter.tensor(Bias_Index)()

    Conv2d_Kernel_size_row = Para[0] #卷积核大小
    Conv2d_Kernel_size_col = Para[1] #卷积核大小
    Conv2d_Kernel_channel = Para[2] #卷积核通道
    Conv2d_Kernel_number = Para[3] #卷积核数量
    Conv2d_bias_number = Para[4]  #偏置数量

    #tensorflow hwc -> chw
    loadData0 = np.swapaxes(Weights, 2, 3)
    loadData2 = np.swapaxes(loadData0, 1, 2)
    #loadData2 = np.swapaxes(loadData1, 0, 1)
    txtfile0 = open(weights_file_path, 'w',encoding='UTF-8')
    #.h File Header
    txtfile0.write("#ifndef __L%d_DWCONV2D_WEIGHTS_H__\n"%(Layer_Number))
    txtfile0.write("#define __L%d_DWCONV2D_WEIGHTS_H__\n"%(Layer_Number))
    txtfile0.write("#include \"opennna_core.h\"\n")
    #数组
    txtfile0.write("const Weights_t L%d_DwConv2d_Weights[%d][%d][%d][%d]={\n"%(Layer_Number,Conv2d_Kernel_number,Conv2d_Kernel_channel,Conv2d_Kernel_size_row,Conv2d_Kernel_size_col))
    #Weights
    for i in range(0,Conv2d_Kernel_number):
        txtfile0.write("\t{\n")
        for j in range(0,Conv2d_Kernel_channel):
            txtfile0.write("\t\t{\n")
            for k in range(0,Conv2d_Kernel_size_row):
                txtfile0.write("\t\t\t{")
                for w in range(0,Conv2d_Kernel_size_col):
                   txtfile0.write(data_t%(loadData2[i][j][k][w])+", ") 
                txtfile0.write("},\n")  
            txtfile0.write("\t\t},\n")
        txtfile0.write("\t},\n")
    #.h File End
    txtfile0.write("\n};")
    txtfile0.write("\n#endif")
    txtfile0.close()
    #Bias
    txtfile1 = open(bias_file_path, 'w',encoding='UTF-8')
    #.h File Header
    txtfile1.write("#ifndef __L%d_DWCONV2D_BIAS_H__\n"%(Layer_Number))
    txtfile1.write("#define __L%d_DWCONV2D_BIAS_H__\n"%(Layer_Number))
    txtfile1.write("#include \"opennna_core.h\"\n")
    #数组
    txtfile1.write("const Bias_t L%d_DwConv2d_Bias[%d]={\n"%(Layer_Number,Conv2d_Kernel_channel))
    txtfile1.write("\t")
    #Bias
    for i in range(0,Conv2d_bias_number):
        txtfile1.write(data_t%(Bias[i])+", ")
    #.h File End
    txtfile1.write("\n};")
    txtfile1.write("\n#endif")
    txtfile1.close()
    print("OpenNNA: Depthwise Conv2d Layer:%d Weights[%d]+Bias[%d] Get Success!"%(Layer_Number,Weights_Index,Bias_Index))

## 10.从Tensorflow(.tflite File)中提取Conv2D的Weights,并生成.h文件
与保存到txt文件不同，保存到.h文件将会同时保存当前层的scale和zero point信息

In [3]:
import numpy as np

#以下为一个提取Tensorflow Conv2d卷积weights+bias的函数，会以txt保存在程序目录下
def OpenNNA_Conv2d_GetWeights_From_TFLite_Save_HFile(Layer_Number,Para,Weights_Index,Bias_Index,precision):
    tflite_interpreter = tf.lite.Interpreter(model_path=tflite_model_name)
    tflite_interpreter.allocate_tensors()
    data_t = precision#"%8.6f"
    weights_file_path = "./L"+str(Layer_Number)+"_Conv2d_Weights"+".h"
    Weights   = tflite_interpreter.tensor(Weights_Index)()
    bias_file_path = "./L"+str(Layer_Number)+"_Conv2d_Bias"+".h"
    Bias = tflite_interpreter.tensor(Bias_Index)()

    Conv2d_Kernel_size_row = Para[0] #卷积核大小
    Conv2d_Kernel_size_col = Para[1] #卷积核大小
    Conv2d_Kernel_channel = Para[2] #卷积核通道
    Conv2d_Kernel_number = Para[3] #卷积核数量
    Conv2d_bias_number = Para[4]  #偏置数量

    #tensorflow hwc -> chw
    loadData0 = np.swapaxes(Weights, 2, 3)
    loadData2 = np.swapaxes(loadData0, 1, 2)
    #loadData2 = np.swapaxes(loadData1, 0, 1)
    txtfile0 = open(weights_file_path, 'w',encoding='UTF-8')
    #.h File Header
    txtfile0.write("#ifndef __L%d_CONV2D_WEIGHTS_H__\n"%(Layer_Number))
    txtfile0.write("#define __L%d_CONV2D_WEIGHTS_H__\n"%(Layer_Number))
    txtfile0.write("#include \"opennna_core.h\"\n")
    #数组
    txtfile0.write("const Weights_t L%d_Conv2d_Weights[%d][%d][%d][%d]={\n"%(Layer_Number,Conv2d_Kernel_number,Conv2d_Kernel_channel,Conv2d_Kernel_size_row,Conv2d_Kernel_size_col))
    #Weights
    for i in range(0,Conv2d_Kernel_number):
        txtfile0.write("\t{\n")
        for j in range(0,Conv2d_Kernel_channel):
            txtfile0.write("\t\t{\n")
            for k in range(0,Conv2d_Kernel_size_row):
                txtfile0.write("\t\t\t{")
                for w in range(0,Conv2d_Kernel_size_col):
                   txtfile0.write(data_t%(loadData2[i][j][k][w])+", ") 
                txtfile0.write("},\n")  
            txtfile0.write("\t\t},\n")
        txtfile0.write("\t},\n")
    #.h File End
    txtfile0.write("\n};")
    txtfile0.write("\n#endif")
    txtfile0.close()
    #Bias
    txtfile1 = open(bias_file_path, 'w',encoding='UTF-8')
    #.h File Header
    txtfile1.write("#ifndef __L%d_CONV2D_BIAS_H__\n"%(Layer_Number))
    txtfile1.write("#define __L%d_CONV2D_BIAS_H__\n"%(Layer_Number))
    txtfile1.write("#include \"opennna_core.h\"\n")
    #数组
    txtfile1.write("const Bias_t L%d_Conv2d_Bias[%d]={\n"%(Layer_Number,Conv2d_Kernel_number))
    txtfile1.write("\t")
    for i in range(0,Conv2d_bias_number):
        txtfile1.write(data_t%(Bias[i])+", ")
    #.h File End
    txtfile1.write("\n};")
    txtfile1.write("\n#endif")
    txtfile1.close()
    print("OpenNNA: Conv2d Layer:%d Weights[%d]+Bias[%d] Get Success!"%(Layer_Number,Weights_Index,Bias_Index))

In [4]:
#与Tensorflow H5 weights&bias的index需要手动printf检查不同， tflite每层weights&bias的index就是Netron所显示的location
OpenNNA_Conv2d_GetWeights_From_TFLite_Save_HFile(0,[3,3,3,8,8],8,6,"%d")#第0层Conv2d

OpenNNA_DWConv2d_GetWeights_From_TFLite_Save_HFile(1,[3,3,8,1,8],35,34,"%d")#第1层Depthwise Conv2d
OpenNNA_Conv2d_GetWeights_From_TFLite_Save_HFile(2,[1,1,8,16,16],38,36,"%d")#第2层Conv2d

OpenNNA_DWConv2d_GetWeights_From_TFLite_Save_HFile(3,[3,3,16,1,16],41,40,"%d")#第3层Depthwise Conv2d
OpenNNA_Conv2d_GetWeights_From_TFLite_Save_HFile(4,[1,1,16,32,32],44,42,"%d")#第4层Conv2d

OpenNNA_DWConv2d_GetWeights_From_TFLite_Save_HFile(5,[3,3,32,1,32],47,46,"%d")#第5层Depthwise Conv2d
OpenNNA_Conv2d_GetWeights_From_TFLite_Save_HFile(6,[1,1,32,32,32],50,48,"%d")#第6层Conv2d

OpenNNA_DWConv2d_GetWeights_From_TFLite_Save_HFile(7,[3,3,32,1,32],53,52,"%d")#第7层Depthwise Conv2d
OpenNNA_Conv2d_GetWeights_From_TFLite_Save_HFile(8,[1,1,32,64,64],56,54,"%d")#第8层Conv2d

OpenNNA_DWConv2d_GetWeights_From_TFLite_Save_HFile(9,[3,3,64,1,64],59,58,"%d")#第9层Depthwise Conv2d
OpenNNA_Conv2d_GetWeights_From_TFLite_Save_HFile(10,[1,1,64,64,64],62,60,"%d")#第10层Conv2d

OpenNNA_DWConv2d_GetWeights_From_TFLite_Save_HFile(11,[3,3,64,1,64],65,64,"%d")#第11层Depthwise Conv2d
OpenNNA_Conv2d_GetWeights_From_TFLite_Save_HFile(12,[1,1,64,128,128],68,66,"%d")#第12层Conv2d

OpenNNA_DWConv2d_GetWeights_From_TFLite_Save_HFile(13,[3,3,128,1,128],71,70,"%d")#第13层Depthwise Conv2d
OpenNNA_Conv2d_GetWeights_From_TFLite_Save_HFile(14,[1,1,128,128,128],74,72,"%d")#第14层Conv2d

OpenNNA_DWConv2d_GetWeights_From_TFLite_Save_HFile(15,[3,3,128,1,128],77,76,"%d")#第15层Depthwise Conv2d
OpenNNA_Conv2d_GetWeights_From_TFLite_Save_HFile(16,[1,1,128,128,128],80,78,"%d")#第16层Conv2d

OpenNNA_DWConv2d_GetWeights_From_TFLite_Save_HFile(17,[3,3,128,1,128],83,82,"%d")#第17层Depthwise Conv2d
OpenNNA_Conv2d_GetWeights_From_TFLite_Save_HFile(18,[1,1,128,128,128],86,84,"%d")#第18层Conv2d

OpenNNA_DWConv2d_GetWeights_From_TFLite_Save_HFile(19,[3,3,128,1,128],11,10,"%d")#第19层Depthwise Conv2d
OpenNNA_Conv2d_GetWeights_From_TFLite_Save_HFile(20,[1,1,128,128,128],14,12,"%d")#第20层Conv2d

OpenNNA_DWConv2d_GetWeights_From_TFLite_Save_HFile(21,[3,3,128,1,128],17,16,"%d")#第21层Depthwise Conv2d
OpenNNA_Conv2d_GetWeights_From_TFLite_Save_HFile(22,[1,1,128,128,128],20,18,"%d")#第22层Conv2d

OpenNNA_DWConv2d_GetWeights_From_TFLite_Save_HFile(23,[3,3,128,1,128],23,22,"%d")#第23层Depthwise Conv2d
OpenNNA_Conv2d_GetWeights_From_TFLite_Save_HFile(24,[1,1,128,256,256],26,24,"%d")#第24层Conv2d

OpenNNA_DWConv2d_GetWeights_From_TFLite_Save_HFile(25,[3,3,256,1,256],29,28,"%d")#第25层Depthwise Conv2d
OpenNNA_Conv2d_GetWeights_From_TFLite_Save_HFile(26,[1,1,256,256,256],32,30,"%d")#第26层Conv2d

#第27层AvgPool

OpenNNA_Conv2d_GetWeights_From_TFLite_Save_HFile(28,[1,1,256,1001,1001],3,2,"%d")#第28层Conv2d

#Output:1001

OpenNNA: Conv2d Layer:0 Weights[8]+Bias[6] Get Success!
OpenNNA: Depthwise Conv2d Layer:1 Weights[35]+Bias[34] Get Success!
OpenNNA: Conv2d Layer:2 Weights[38]+Bias[36] Get Success!
OpenNNA: Depthwise Conv2d Layer:3 Weights[41]+Bias[40] Get Success!
OpenNNA: Conv2d Layer:4 Weights[44]+Bias[42] Get Success!
OpenNNA: Depthwise Conv2d Layer:5 Weights[47]+Bias[46] Get Success!
OpenNNA: Conv2d Layer:6 Weights[50]+Bias[48] Get Success!
OpenNNA: Depthwise Conv2d Layer:7 Weights[53]+Bias[52] Get Success!
OpenNNA: Conv2d Layer:8 Weights[56]+Bias[54] Get Success!
OpenNNA: Depthwise Conv2d Layer:9 Weights[59]+Bias[58] Get Success!
OpenNNA: Conv2d Layer:10 Weights[62]+Bias[60] Get Success!
OpenNNA: Depthwise Conv2d Layer:11 Weights[65]+Bias[64] Get Success!
OpenNNA: Conv2d Layer:12 Weights[68]+Bias[66] Get Success!
OpenNNA: Depthwise Conv2d Layer:13 Weights[71]+Bias[70] Get Success!
OpenNNA: Conv2d Layer:14 Weights[74]+Bias[72] Get Success!
OpenNNA: Depthwise Conv2d Layer:15 Weights[77]+Bias[76] G

### 将ImageNet的.jpg转换为c 数组存到.txt中

In [5]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import models, layers, optimizers
import numpy as np

channel = 3
row = 128
col = 128

image_value = tf.io.read_file('./imagenet/tiger.jpg')

#解码为tensor
image_value = tf.io.decode_jpeg(image_value,channels = 3)


image_value = tf.image.resize(image_value, (row,col))#改变像素值为128*128


#tensor转array
image_value = image_value.numpy()


loadData1 = np.swapaxes(image_value, 0, 2)
loadData2 = np.swapaxes(loadData1, 1, 2)


txtfile = open(r'ILSVRC2012_val_00000005_uint8.txt', 'w',encoding='UTF-8')
for i in range(0,channel):
    txtfile.write("\t{\n")
    for j in range(0,row):
        txtfile.write("\t\t{")
        for k in range(0,col):
            #txtfile.write(str(loadData2[i][j][k])+", ") 
            txtfile.write("%d"%(loadData2[i][j][k])+", ") 
        txtfile.write("},\n")
    txtfile.write("\t},\n\n")
txtfile.close()